In [ ]:
import pandas as pd
import numpy as np
import re
from shapely.geometry import Point, Polygon
import geopandas as gpd

##### Leer capas pasadas 

In [ ]:
PATH = input('Please enter the path where are you working')

In [ ]:
PATH = PATH + '\\'

In [ ]:
last_geo_file = 'pv_poligonos.geojson'

In [ ]:
last_geojson_file_poly = PATH+last_geo_file

In [ ]:
last_geodf = gpd.read_file(last_geojson_file_poly)

In [ ]:
last_geodf.head(2)

In [ ]:
last_geodf.info()

In [ ]:
# Sí existe
last_geo_file_points = 'pv_puntos.geojson'
last_geojson_file_points = PATH+last_geo_file_points


In [ ]:
last_geodf_points = gpd.read_file(last_geojson_file_points)


In [ ]:

last_geodf_points.head(2)

In [ ]:
last_geodf_points.info()

In [ ]:
cols=list(last_geodf.columns)

In [ ]:
cols

In [ ]:
cols_points = list(last_geodf_points.columns)
cols_points

##### Formato ciclo a reportar

In [ ]:
def to_float(i):
    """
    el parámetro i es un del tipo str
    -----------------------------------
    La función to_float convierte a 'i' a tipo flotante
    -----------------------------------
    devuelve un valor del tipo float
    
    """
    i = float(i)
    return i

In [ ]:
def transform(row):
    """
    el parámetro row es un valor del tipo str
    ----------------------------------------------
    separa los elementos de cada row de acuerdo a ciertos caracteres
    depura aquellos que no cumplen ciertas condiciones asignandoles el valor de lista vacia
    transforma cada cadena de texto a valores del tipo float
    ------------------------------------------------
    Retorna coor la cual puede ser una lista con valores flotantes 
    o una lista vacía.

    """
    input = row.split(',')

    size_list = len(input)


    if size_list ==1:
        coor = []
    elif len(input) ==2:
        try:
            pre_punto = map(to_float,input)
            coor=list(pre_punto)
        except ValueError as e:
            print(e)
    elif len(input) >2:
        not_point = re.split(r",0\\n\\n|,0|0\s-|,",row)
        if ("" in not_point):
            not_point.remove("")
            list_with_float = map(to_float,not_point)
            coor = list(list_with_float)
        else:
            list_with_float = map(to_float,not_point)
            coor = list(list_with_float)
    return coor

In [ ]:
def arreglo(listado):

    ar = listado.sort()
    return ar

In [ ]:
def to_matrix(listado):
    """
    el parámetro listado es un objeto del tipo lista
    -----------------------------------
    transforma una lista a un array de numpy
    asigna la forma del array
    -----------------------------------
    devuelve una matriz
    """
    n = len(listado)
    c = 2
    f =  n/c
    f = int(f)
    vector = np.array(listado)
    matriz = vector.reshape(f,c)
    return matriz

In [ ]:
def geometry_apply(x):
    """
    el parámetro x es una lista
    ------------------------------------------
    de acuerdo a la longitud de la lista aplica una o dos funciones
    -----------------------------------------
    devuelve geometry que puede ser un objeto Point, Polygon o un valor None
    """

    n = x

    if len(n) == 2:
        # orden=arreglo(n)
        # n = n.sort()
        geometry = Point(n)
    elif len(n) >5 and len(n) %2 == 0: #Nueva validación
        listado = to_matrix(n)
        geometry = Polygon(listado)
    else:
        geometry = None
    print(geometry)
    return geometry

In [ ]:
def typeshape(x):
    """
    El parámetro x es un str
    ----------------------------------------
    De acuerdo al valor genera ciertas cadenas de texto
    ---------------------------------------
    retorna label que es un str
    """
    n=x
    if int(n) == 2:
        label = "Point"
    else:
        label = "Polygon"
    return label    

In [ ]:
target_file = 'parcelas.csv'

In [ ]:
# crear archivos de salida
puntos = PATH+'points.geojson'
poligonos = PATH+'polygons.geojson'

In [ ]:
def main(file_input):
    """
    file_input recibe al archivo Sitios.txt
    -----------------------------------------------
    Lee el archivo
    genera columnas y aplica funciones
    crea 3 subsets distintos
    elimina columnas temporales y columna Coordenadas areas
    exportar los sets a un formato específico
    -----------------------------------------------
    genera los archivos de salida o un error
    """
    try:
        sitios= pd.read_csv(file_input)
        sitiosbd = sitios.copy()
        sitiosbd['info_geo'] = sitiosbd['Coordenadas.areas']
        sitiosbd['info_geo']=sitiosbd['info_geo'].astype(str)
        sitiosbd['info_geo']= sitiosbd['info_geo'].apply(transform)
        desechados=sitiosbd[sitiosbd['info_geo'].isnull()]
        print(desechados.shape)
        sitiosbd_depurate=sitiosbd[sitiosbd['info_geo'].notna()]
        sitiosbd_depurate['n_vertices'] = sitiosbd_depurate['info_geo'].apply(lambda x:len(x))
        # sitiosbd_depurate.to_csv(f'{PATH}n_vertices.csv')
        # sitiosbd_depurate['type_geometry'] = sitiosbd_depurate['info_geo'].apply(arreglo)
        sitiosbd_depurate['type_geometry'] = sitiosbd_depurate['info_geo'].apply(geometry_apply)
        desechados_2=sitiosbd_depurate[sitiosbd_depurate['type_geometry'].isnull()]
        desechados = pd.concat([desechados_2,desechados],axis=1)
        sitiosbd_depurate=sitiosbd_depurate[sitiosbd_depurate['type_geometry'].notna()]
        desechados.to_csv(f'{PATH}errors.csv')
        sitiosbd_depurate['type_shape'] = sitiosbd_depurate['n_vertices'].apply(typeshape)
        sitiosbd_polygon = sitiosbd_depurate[sitiosbd_depurate['type_shape']=='Polygon']
        sitiosbd_point = sitiosbd_depurate[sitiosbd_depurate['type_shape']=='Point']
        gdf_points_sitios = gpd.GeoDataFrame(sitiosbd_point,geometry='type_geometry',crs='CRS84')
        gdf_polygons_sitios = gpd.GeoDataFrame(sitiosbd_polygon,geometry='type_geometry',crs='CRS84')
 #el formato GeoJSON no soporta el formato de listas(un arreglo)
        eliminar = ['Coordenadas.areas','info_geo','n_vertices','type_shape','Nombre.Productor','Bitacora.registrada.por']
        gdf_points_sitios=gdf_points_sitios.drop(columns=eliminar)
        gdf_polygons_sitios=gdf_polygons_sitios.drop(columns=eliminar)

        gdf_points_sitios.to_file(puntos,crs='CRS84', driver='GeoJSON')
        gdf_polygons_sitios.to_file(poligonos,crs='CRS84', driver='GeoJSON')
    except FileNotFoundError as e:
        print(f'Por favor ejecute el script en el mismo directorio donde se encuentre el archivo Sitio.txt {e}')

In [ ]:
if __name__ == "__main__":
    main(f'{PATH}{target_file}')

##### Ciclo a Reportar (GeoDF)

In [ ]:
new_geodf = gpd.read_file(poligonos)

In [ ]:
new_geodf.head(2)

In [ ]:
new_geodf.info()

In [ ]:
new_geodf_points = gpd.read_file(puntos)

In [ ]:
new_geodf_points.head(2)

In [ ]:
new_geodf_points.info()

In [ ]:
cols_new_geo = list(new_geodf.columns)

In [ ]:
cols_new_geo

In [ ]:
cols_new_geo_points = list(new_geodf_points.columns)

In [ ]:
cols_new_geo_points

##### Consolidado

###### Poligonos

In [ ]:
def cols_no_estan(lista_cols_pasada, lista_cols_nueva,dataset):
    try:
        dataset = dataset[lista_cols_pasada]
        return dataset
    except KeyError as e:
        print(e)
        return [name for name in lista_cols_pasada if name not in lista_cols_nueva]


In [ ]:
cols_no_estan(cols,cols_new_geo,new_geodf)

In [ ]:
cols_new_geo

In [ ]:
new_geodf.rename(columns={'ID.de.bitacora':'ID','Cultivo.1':'Cultivo'},inplace=True)

In [ ]:
salida_polygons=cols_no_estan(cols,cols_new_geo,new_geodf)

In [ ]:
salida_polygons.info()

In [ ]:
last_geodf.info()

In [ ]:
# emperejamos la proyeccion 
salida_polygons.crs

In [ ]:
last_geodf.crs

In [ ]:
salida_polygons.to_crs('EPSG:4326',inplace=True)

In [ ]:
salida_polygons.crs

In [ ]:
polygonos_pv = last_geodf.append(salida_polygons,ignore_index=True)

In [ ]:
polygonos_pv.info()

In [ ]:
polygonos_pv.drop_duplicates(keep='last',inplace=True)

In [ ]:
polygonos_pv.info()

In [ ]:
def clean_text(string):
    lista=string.split('_')
    if len(lista) > 1:
        lista.pop(0)
        municipio = ' '.join(lista)
        return municipio
    else:
        municipio = string

        return municipio

In [ ]:
polygonos_pv['Municipio.areas'].tail()

In [ ]:
polygonos_pv['Municipio.areas'].head()

In [ ]:
polygonos_pv['Municipio.areas']= polygonos_pv['Municipio.areas'].apply(clean_text)

In [ ]:
polygonos_pv['Municipio.areas'].tail()

In [ ]:
polygonos_pv['Municipio.areas'].head()

In [ ]:
polygonos_pv['Tipo.de.bitacora'] = polygonos_pv['Tipo.de.bitacora'].replace('AE','AREA DE EXTENSION')

In [ ]:
polygonos_pv['Proyecto'] = polygonos_pv['Proyecto'].replace('AGUAS_FIRMES_APAN','AGUAS FIRMES APAN')

In [ ]:
polygonos_pv['Proyecto'].tail()

In [ ]:
polygonos_pv['Tipo.de.bitacora'].head()

In [ ]:
target=['Tipo.de.bitacora','Regimen.hidrico','Proyecto','Acuifero']

In [ ]:
for col in target:
    polygonos_pv[col] = polygonos_pv[col].str.title()

In [ ]:
polygonos_pv['Tipo.de.bitacora']

In [ ]:
polygonos_pv.to_file('polygonos_pv22.geojson',crs='EPSG:4326', driver='GeoJSON')

###### Puntos

In [ ]:
cols_new_geo_points

In [ ]:
cols_no_estan(cols_points,cols_new_geo_points,new_geodf_points)

In [ ]:
cols_points

In [ ]:
last_geodf_points=last_geodf_points[[
 'ID.parcela',
 'ID.del.sitio',
 'Estado.areas',
 'Municipio.areas',
 'ID.de.bitacora',
 'Tipo.de.bitacora',
 'Tipo.de.superficie',
 'Anio.Bitacora',
 'Ciclo',
 'Regimen.hidrico',
 'Superficie.sembrada',
 'Superficie.Total.de.la.parcela',
 'Cultivo',
 'Proyecto',
 'Acuifero',
 'geometry']]

In [ ]:
cols_points = list(last_geodf_points.columns)
cols_points

In [ ]:
cols_no_estan(cols_points,cols_new_geo_points,new_geodf_points)

In [ ]:
cols_new_geo_points

In [ ]:
new_geodf_points.rename(columns={'Cultivo.1':'Cultivo'},inplace=True)

In [ ]:
salida_puntos=cols_no_estan(cols_points,cols_new_geo_points,new_geodf_points)

In [ ]:
salida_puntos.info()

In [ ]:
salida_puntos.crs

In [ ]:
last_geodf_points.crs

In [ ]:
salida_puntos.to_crs('EPSG:4326',inplace=True)

In [ ]:
puntos_pv22 = last_geodf_points.append(salida_puntos,ignore_index=True)

In [ ]:
puntos_pv22.info()

In [ ]:
puntos_pv22['Municipio.areas'] = puntos_pv22['Municipio.areas'].apply(clean_text)

In [ ]:
puntos_pv22['Municipio.areas'].tail(20)

In [ ]:
puntos_pv22['Tipo.de.bitacora']=puntos_pv22['Tipo.de.bitacora'].replace('AI simplificada','Área de Impacto Simplificada')

In [ ]:
puntos_pv22['Tipo.de.bitacora']=puntos_pv22['Tipo.de.bitacora'].replace('AI','Área de Impacto')

In [ ]:
puntos_pv22['Tipo.de.bitacora']

In [ ]:
puntos_pv22['Proyecto']=puntos_pv22['Proyecto'].replace('AGUAS_FIRMES_APAN','Aguas Firmes Apan')

In [ ]:
puntos_pv22['Proyecto']

In [ ]:
puntos_pv22['Proyecto'] = puntos_pv22['Proyecto'].str.title()

In [ ]:
puntos_pv22['Proyecto'] 

In [ ]:
puntos_pv22['Acuifero'] = puntos_pv22['Acuifero'].str.title()

In [ ]:
puntos_pv22['Acuifero'] 

In [ ]:
puntos_pv22.drop_duplicates(keep='last',inplace=True)

In [ ]:
puntos_pv22.info()

In [ ]:
puntos_pv22.to_file('puntos_pv22.geojson',crs='EPSG:4326', driver='GeoJSON')